In [2]:
import pandas as pd
import numpy as np

# Load Results

In [3]:
RESULTS_FILE = './results.csv'
NUMBER_OF_EXPERIMENTS = 27 * 228
PARAM_COLUMNS = ['fitness_function', 'use_constraint', 'constraint_handling_method', 'constraint',
                 'particle_speed_limit', 'population_size', 'personal_confidence', 'swarm_confidence',
                 'particle_inertia', 'constraint_r']
RESULT_COLUMNS_ALL = ['fitness', 'optimum', 'iterations', 'optimum_reached', 'iterations_to_opt']#
RESULT_COLUMNS = ['fitness', 'optimum', 'optimum_reached', 'iterations_to_opt']

In [4]:
df_all_iterations = pd.read_csv(RESULTS_FILE)
df_all_iterations.head()

,number,iteration,fitness_function,use_constraint,constraint_handling_method,constraint,particle_speed_limit,population_size,personal_confidence,swarm_confidence,particle_inertia,constraint_r,fitness,optimum,iterations,optimum_reached,iterations_to_opt
0,10001,1,Shubert function,False,NaN,NaN,13,5,0.5,1.5,0.1,-0.15,0.749117,1.0,500,False,9
1,10002,1,Shubert function,False,NaN,NaN,13,5,1.0,1.0,0.1,-0.15,0.641035,1.0,500,False,33
2,10003,1,Shubert function,False,NaN,NaN,13,5,1.5,0.5,0.1,-0.15,0.698651,1.0,500,False,0
3,10004,1,Shubert function,False,NaN,NaN,13,5,0.5,1.5,0.5,-0.15,0.749117,1.0,500,False,15
4,10005,1,Shubert function,False,NaN,NaN,13,5,1.0,1.0,0.5,-0.15,0.749117,1.0,500,False,15


# Experiment Parameters

In [5]:
df_params = df_all_iterations[df_all_iterations.iteration == 1].set_index('number')[PARAM_COLUMNS]
df_params.constraint_r.replace(-0.15, np.nan, inplace=True)  # default was used when no constraint used -> change to NAN
assert df_params.index.is_unique
assert df_params.shape == (NUMBER_OF_EXPERIMENTS, len(PARAM_COLUMNS))
df_params.head()

,fitness_function,use_constraint,constraint_handling_method,constraint,particle_speed_limit,population_size,personal_confidence,swarm_confidence,particle_inertia,constraint_r
number,,,,,,,,,,
10001,Shubert function,False,NaN,NaN,13,5,0.5,1.5,0.1,NaN
10002,Shubert function,False,NaN,NaN,13,5,1.0,1.0,0.1,NaN
10003,Shubert function,False,NaN,NaN,13,5,1.5,0.5,0.1,NaN
10004,Shubert function,False,NaN,NaN,13,5,0.5,1.5,0.5,NaN
10005,Shubert function,False,NaN,NaN,13,5,1.0,1.0,0.5,NaN


In [6]:
# sanity check params
assert set(df_params.fitness_function.unique()) == set(['Shubert function', "Booth's function", 'Schwefel function'])#
assert set(df_params.use_constraint.unique()) == set([True, False])
assert set(df_params.constraint_handling_method.unique()) == set([np.nan, 'Penalty Method', 'Rejection Method'])
assert set(df_params.constraint.unique()) == set([np.nan, 'Constraint 3', 'Constraint 5', 'Constraint 10'])
# assert set(df_params.particle_speed_limit.unique()) == set([2, 10, 19])
assert df_params.particle_speed_limit.unique() == [13] # redo some experiments with variation
assert set(df_params.population_size.unique()) == set([5, 10, 35, 80])
assert set(df_params.personal_confidence.unique()) == set([0.5, 1.0, 1.5])
assert set(df_params.swarm_confidence.unique()) == set([0.5, 1.0, 1.5])
assert set(df_params.particle_inertia.unique()) == set([0.1, 0.5, 0.9])
assert set(df_params.constraint_r.dropna().unique()) == set([-1.0, -1.5, -2.0])
assert df_params[~df_params.use_constraint][['constraint_handling_method', 'constraint', 'constraint_r']].isna().all(axis=None)
assert df_params[df_params.use_constraint][['constraint_handling_method', 'constraint', 'constraint_r']].notna().all(axis=None)

In [7]:
# inspect values in Experiment setup
for column in PARAM_COLUMNS:
    print(f'{column}: {df_params[column].unique()}')

fitness_function: ['Shubert function' "Booth's function" 'Schwefel function']
use_constraint: [False  True]
constraint_handling_method: [nan 'Penalty Method' 'Rejection Method']
constraint: [nan 'Constraint 3' 'Constraint 5' 'Constraint 10']
particle_speed_limit: [13]
population_size: [ 5 10 35 80]
personal_confidence: [0.5 1.  1.5]
swarm_confidence: [1.5 1.  0.5]
particle_inertia: [0.1 0.5 0.9]
constraint_r: [ nan -1.  -1.5 -2. ]


# Experiment Results

In [8]:
df_results = df_all_iterations[['number'] + RESULT_COLUMNS_ALL]
df_results = df_results.groupby('number').mean()
## sanity check results
assert df_results.shape == (NUMBER_OF_EXPERIMENTS, len(RESULT_COLUMNS_ALL))
assert df_results.fitness.min() >= 0.0
assert df_results.fitness.max() <= 1.0
assert df_results.optimum.min() > 0.0
assert df_results.optimum.max() <= 1.0
assert df_results.iterations.unique() == [500.0]
assert df_results.optimum_reached.min() >= 0.0
assert df_results.optimum_reached.max() <= 1.0
assert df_results.iterations_to_opt.min() >= 0
assert df_results.iterations_to_opt.max() <= 500
df_results.head()

,fitness,optimum,iterations,optimum_reached,iterations_to_opt
number,,,,,
10001,0.685586,1.0,500.0,0.0,8.5
10002,0.632124,1.0,500.0,0.0,144.0
10003,0.700413,1.0,500.0,0.0,5.0
10004,0.686165,1.0,500.0,0.0,14.0
10005,0.686165,1.0,500.0,0.0,21.5


In [9]:
df_results[RESULT_COLUMNS].describe()

,fitness,optimum,optimum_reached,iterations_to_opt
count,6156.000000,6156.000000,6156.000000,6156.000000
mean,0.896854,0.930320,0.550520,39.536712
std,0.158699,0.152377,0.466556,65.502258
min,0.471753,0.495829,0.000000,0.000000
25%,0.874587,0.924913,0.000000,8.000000
50%,0.984247,1.000000,0.500000,16.000000
75%,1.000000,1.000000,1.000000,38.500000
max,1.000000,1.000000,1.000000,498.000000


# Combined Data by Experiment

In [10]:
df = df_params.join(df_results)
assert df.shape == (NUMBER_OF_EXPERIMENTS, len(PARAM_COLUMNS) + len(RESULT_COLUMNS_ALL))
df.head()

,fitness_function,use_constraint,constraint_handling_method,constraint,particle_speed_limit,population_size,personal_confidence,swarm_confidence,particle_inertia,constraint_r,fitness,optimum,iterations,optimum_reached,iterations_to_opt
number,,,,,,,,,,,,,,,
10001,Shubert function,False,NaN,NaN,13,5,0.5,1.5,0.1,NaN,0.685586,1.0,500.0,0.0,8.5
10002,Shubert function,False,NaN,NaN,13,5,1.0,1.0,0.1,NaN,0.632124,1.0,500.0,0.0,144.0
10003,Shubert function,False,NaN,NaN,13,5,1.5,0.5,0.1,NaN,0.700413,1.0,500.0,0.0,5.0
10004,Shubert function,False,NaN,NaN,13,5,0.5,1.5,0.5,NaN,0.686165,1.0,500.0,0.0,14.0
10005,Shubert function,False,NaN,NaN,13,5,1.0,1.0,0.5,NaN,0.686165,1.0,500.0,0.0,21.5


# Evaluation

## Confidence

In [17]:
df[~df.use_constraint].groupby(['fitness_function', 'personal_confidence', 'swarm_confidence'])[RESULT_COLUMNS].mean().round(3).astype(str) + '+/-' + df[~df.use_constraint].groupby(['fitness_function', 'personal_confidence', 'swarm_confidence'])[RESULT_COLUMNS].std().round(3).astype(str)

fitness  \
fitness_function  personal_confidence swarm_confidence                  
Booth's function  0.5                 1.5               0.999+/-0.004   
                  1.0                 1.0               0.999+/-0.003   
                  1.5                 0.5               0.999+/-0.002   
Schwefel function 0.5                 1.5               0.954+/-0.058   
                  1.0                 1.0               0.931+/-0.087   
                  1.5                 0.5                0.913+/-0.09   
Shubert function  0.5                 1.5               0.843+/-0.141   
                  1.0                 1.0                0.87+/-0.142   
                  1.5                 0.5               0.902+/-0.117   

                                                          optimum  \
fitness_function  personal_confidence swarm_confidence              
Booth's function  0.5                 1.5               1.0+/-0.0   
                  1.0                 1.0               1.0+/-0.0   
                  1.5                 0.5               1.0+/-0.0   
Schwefel function 0.5                 1.5               1.0+/-0.0   
                  1.0                 1.0               1.0+/-0.0   
                  1.5                 0.5               1.0+/-0.0   
Shubert function  0.5                 1.5               1.0+/-0.0   
                  1.0                 1.0               1.0+/-0.0   
                  1.5                 0.5               1.0+/-0.0   

                                                       optimum_reached  \
fitness_function  personal_confidence swarm_confidence                   
Booth's function  0.5                 1.5                0.833+/-0.316   
                  1.0                 1.0                0.833+/-0.378   
                  1.5                 0.5                0.792+/-0.385   
Schwefel function 0.5                 1.5                0.625+/-0.469   
                  1.0                 1.0                  0.583+/-0.5   
                  1.5                 0.5                  0.417+/-0.5   
Shubert function  0.5                 1.5                  0.5+/-0.463   
                  1.0                 1.0                 0.625+/-0.42   
                  1.5                 0.5                  0.5+/-0.463   

                                                       iterations_to_opt  
fitness_function  personal_confidence swarm_confidence                    
Booth's function  0.5                 1.5                 13.792+/-9.735  
                  1.0                 1.0                14.167+/-11.452  
                  1.5                 0.5                29.375+/-27.799  
Schwefel function 0.5                 1.5                84.25+/-110.925  
                  1.0                 1.0                80.917+/-64.255  
                  1.5                 0.5                69.917+/-66.673  
Shubert function  0.5                 1.5                15.167+/-13.768  
                  1.0                 1.0                 26.75+/-36.361  
                  1.5                 0.5                111.0+/-150.317

# Correlation

In [13]:
df.drop(columns=["use_constraint", "iterations"]).corr()

C:\Users\karst\AppData\Local\Temp\ipykernel_10864\2416084564.py:1: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  df.drop(columns=["use_constraint", "iterations"]).corr()


,particle_speed_limit,population_size,personal_confidence,swarm_confidence,particle_inertia,constraint_r,fitness,optimum,optimum_reached,iterations_to_opt
particle_speed_limit,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
population_size,NaN,1.000000e+00,2.245036e-15,-6.410103e-16,-9.345388e-18,4.087378e-15,0.152972,-3.012438e-15,0.382062,-0.186107
personal_confidence,NaN,2.245036e-15,1.000000e+00,-1.000000e+00,2.434700e-18,-1.553398e-17,-0.006594,1.699272e-16,-0.074630,0.129656
swarm_confidence,NaN,-6.410103e-16,-1.000000e+00,1.000000e+00,1.352611e-18,1.050828e-17,0.006594,2.464639e-16,0.074630,-0.129656
particle_inertia,NaN,-9.345388e-18,2.434700e-18,1.352611e-18,1.000000e+00,-6.282126e-18,0.061281,9.439959e-18,0.386373,-0.027106
constraint_r,NaN,4.087378e-15,-1.553398e-17,1.050828e-17,-6.282126e-18,1.000000e+00,0.018253,8.667159e-17,0.008099,-0.056173
fitness,NaN,1.529724e-01,-6.593725e-03,6.593725e-03,6.128127e-02,1.825301e-02,1.000000,8.770338e-01,0.196999,-0.016953
optimum,NaN,-3.012438e-15,1.699272e-16,2.464639e-16,9.439959e-18,8.667159e-17,0.877034,1.000000e+00,-0.013000,0.043813
optimum_reached,NaN,3.820622e-01,-7.463049e-02,7.463049e-02,3.863727e-01,8.098888e-03,0.196999,-1.299964e-02,1.000000,-0.210113
iterations_to_opt,NaN,-1.861066e-01,1.296556e-01,-1.296556e-01,-2.710574e-02,-5.617346e-02,-0.016953,4.381324e-02,-0.210113,1.000000
